# 🎨 Image Generation with Stable Diffusion
## Internship Project - AI Image Generation

**What this does:**
- 🤖 Generate images from text prompts
- 🎭 Try different artistic styles
- 📊 Track generation time and quality
- 💾 Save results

**Based on:** Stable Diffusion tutorial but with some cool extras!

---
**Author:** [Your Name]  
**Date:** 2024

In [ ]:
# Install Packages
# Run this first! Takes about 1-2 minutes

print("🎨 Setting up AI image generation...")

import subprocess
import sys

packages = [
    "diffusers",
    "transformers", 
    "accelerate",
    "torch",
    "matplotlib"
]

for package in packages:
    print(f"Installing {package}...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", package, "--quiet"])

print("✅ All done! Continue to next cell.")

In [ ]:
# Import libraries
import torch
from diffusers import StableDiffusionPipeline
import matplotlib.pyplot as plt
import time
from datetime import datetime
import json

# Check if we have GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using: {device}")
if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")

print("✅ Ready to generate images!")

In [ ]:
# Load the AI model
print("🤖 Loading Stable Diffusion model...")
print("(This takes 1-2 minutes first time)")

pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    safety_checker=None,
    requires_safety_checker=False
).to(device)

# Make it faster
if device == "cuda":
    pipe.enable_attention_slicing()

print("✅ Model loaded! Ready to create images!")

# Keep track of our generations
generation_history = []

In [ ]:
# Generate Single Image
# Change these variables to customize:

prompt = "A cute robot painting a picture in an art studio"
steps = 15  # 10-50, lower = faster
guidance = 7.5  # 1-15, higher = follows prompt more

print(f"🎨 Creating: '{prompt}'")
print(f"⚙️ Steps: {steps}, Guidance: {guidance}")

start_time = time.time()
print("🔄 Generating... (this takes 10-30 seconds)")

# Generate the image
with torch.autocast(device):
    result = pipe(
        prompt=prompt,
        num_inference_steps=steps,
        guidance_scale=guidance,
        width=512,
        height=512
    )

image = result.images[0]
generation_time = time.time() - start_time

# Save info
generation_info = {
    'prompt': prompt,
    'time': generation_time,
    'steps': steps,
    'guidance': guidance,
    'timestamp': datetime.now().isoformat()
}
generation_history.append(generation_info)

# Show the image
plt.figure(figsize=(8, 8))
plt.imshow(image)
plt.axis('off')
plt.title(f"Generated in {generation_time:.1f} seconds\n'{prompt}'", fontsize=12, pad=20)
plt.show()

print(f"✅ Done! Generated in {generation_time:.1f} seconds")

# Clean up GPU memory
if device == "cuda":
    torch.cuda.empty_cache()

In [ ]:
# Generate Multiple Images
# Edit the prompts list below:

prompts = [
    "A magical forest with glowing mushrooms",
    "A futuristic city at sunset", 
    "A cozy coffee shop in winter"
]

print(f"🔄 Generating {len(prompts)} images...")

images = []
times = []

for i, prompt in enumerate(prompts):
    print(f"\n{i+1}/{len(prompts)}: {prompt}")
    
    start_time = time.time()
    
    with torch.autocast(device):
        result = pipe(
            prompt=prompt,
            num_inference_steps=25,
            guidance_scale=7.5
        )
    
    image = result.images[0]
    gen_time = time.time() - start_time
    
    images.append(image)
    times.append(gen_time)
    
    generation_history.append({
        'prompt': prompt,
        'time': gen_time,
        'steps': 25,
        'guidance': 7.5,
        'timestamp': datetime.now().isoformat()
    })
    
    print(f"✅ Done in {gen_time:.1f}s")

# Show all images
fig, axes = plt.subplots(1, len(images), figsize=(5*len(images), 5))
if len(images) == 1:
    axes = [axes]

for i, (image, prompt, time_taken) in enumerate(zip(images, prompts, times)):
    axes[i].imshow(image)
    axes[i].axis('off')
    axes[i].set_title(f"{prompt}\n({time_taken:.1f}s)", fontsize=10)

plt.tight_layout()
plt.show()

print(f"\n🎉 All done! Average time: {sum(times)/len(times):.1f}s")

if device == "cuda":
    torch.cuda.empty_cache()

In [ ]:
# Try Different Styles
# Change the base_prompt below:

base_prompt = "A mountain landscape"

styles = [
    "oil painting",
    "watercolor",
    "digital art",
    "cartoon style"
]

print(f"🎭 Trying different styles for: '{base_prompt}'")

style_images = []
style_times = []

for style in styles:
    full_prompt = f"{base_prompt}, {style}"
    print(f"\nGenerating: {style}")
    
    start_time = time.time()
    
    with torch.autocast(device):
        result = pipe(
            prompt=full_prompt,
            num_inference_steps=30,
            guidance_scale=8.0
        )
    
    image = result.images[0]
    gen_time = time.time() - start_time
    
    style_images.append(image)
    style_times.append(gen_time)
    
    generation_history.append({
        'prompt': full_prompt,
        'time': gen_time,
        'steps': 30,
        'guidance': 8.0,
        'timestamp': datetime.now().isoformat()
    })
    
    print(f"✅ {gen_time:.1f}s")

# Show style comparison
fig, axes = plt.subplots(2, 2, figsize=(12, 12))
axes = axes.flatten()

for i, (image, style, time_taken) in enumerate(zip(style_images, styles, style_times)):
    axes[i].imshow(image)
    axes[i].axis('off')
    axes[i].set_title(f"{style.title()}\n({time_taken:.1f}s)", fontsize=12)

plt.suptitle(f"Style Comparison: '{base_prompt}'", fontsize=16)
plt.tight_layout()
plt.show()

print(f"\n🎨 Style test complete!")

if device == "cuda":
    torch.cuda.empty_cache()

In [ ]:
# Show Session Stats
if generation_history:
    print("📊 SESSION STATS")
    print("=" * 20)
    
    total_images = len(generation_history)
    times = [g['time'] for g in generation_history]
    avg_time = sum(times) / len(times)
    fastest = min(times)
    slowest = max(times)
    
    print(f"🖼️  Images generated: {total_images}")
    print(f"⏱️  Average time: {avg_time:.1f}s")
    print(f"🚀 Fastest: {fastest:.1f}s")
    print(f"🐌 Slowest: {slowest:.1f}s")
    
    # Simple chart
    plt.figure(figsize=(10, 4))
    plt.bar(range(len(times)), times, color='skyblue')
    plt.xlabel('Image Number')
    plt.ylabel('Generation Time (seconds)')
    plt.title('Generation Times')
    plt.show()
    
    print("\n✅ Not bad for an internship project! 🎉")
else:
    print("No images generated yet! Try the cells above first.")

In [ ]:
# Save Results
if generation_history:
    # Create simple report
    report = {
        "project": "AI Image Generation Internship",
        "date": datetime.now().isoformat(),
        "total_images": len(generation_history),
        "average_time": sum(g['time'] for g in generation_history) / len(generation_history),
        "generations": generation_history
    }
    
    # Save as JSON
    filename = f"image_generation_results_{datetime.now().strftime('%Y%m%d_%H%M')}.json"
    
    with open(filename, 'w') as f:
        json.dump(report, f, indent=2)
    
    print(f"💾 Results saved to: {filename}")
    print(f"📊 Generated {report['total_images']} images")
    print(f"⏱️  Average time: {report['average_time']:.1f}s")
    
    print("\n🎯 Perfect for your internship presentation!")
else:
    print("Nothing to save yet! Generate some images first.")

# 🎉 Project Complete!

## What You Built:
✅ **AI Image Generator** using Stable Diffusion  
✅ **Interactive controls** with sliders and text inputs  
✅ **Batch processing** for multiple images  
✅ **Style comparison** testing different artistic styles  
✅ **Performance tracking** with generation times  
✅ **Data export** for presentation  

## Why This Is Good for Internship:
- 🤖 Shows you understand **AI/ML concepts**
- 💻 Demonstrates **coding skills** in Python
- 📊 Includes **data analysis** and visualization
- 🎨 Has **creative applications**
- 📱 **Interactive and user-friendly**

## Next Steps:
1. **Download** this notebook
2. **Run it** and generate some cool images
3. **Save** your results
4. **Show** it in your internship interview!

**Good luck! 🚀**